source: https://www.marcell-dietl.de/downloads/QuadratischesSieb.pdf#page=45&zoom=100,118,418

In [1]:
from sage import all
from itertools import islice, count


In [32]:
number=178691*1785693
print(number)

319087267863


In [33]:
def calculate_boundary (number) :
    return ceil(sqrt(exp(sqrt(log(number)*log(log(number))))))

boundary = calculate_boundary(number)
print(boundary)

106


In [34]:
def generate_factor_base (number , boundary ):
    factor_base=[-1,2]

    for prime in prime_range(3,boundary):
        if legendre_symbol(number, prime)==1:
            factor_base.append(prime)
    return factor_base

factor_base =generate_factor_base(number, boundary)
print(factor_base)


[-1, 2, 11, 17, 43, 47, 53, 59, 73, 83, 97, 101, 103]


In [38]:
def sieve(number, boundary, factor_base, number_of_intervals = 1):
    start_point = ceil(sqrt(number))-(boundary**2) //2
    end_point = start_point+2*(boundary**2)//2

    threshold = ceil(log((start_point+((end_point - start_point)//5)*3)**2 - number, 2) ) - 30

    sieved_array = []
    interval_length = ( end_point - start_point ) // number_of_intervals
    end_point = start_point - 1
    for interval in islice(count(0), number_of_intervals ) :
        start_point = end_point + 1
        end_point = start_point + interval_length
        sieved_array.append( __sieve_interval(number, factor_base,start_point, end_point, threshold))
    return reduce(lambda x , y : x + y , sieved_array )

sieved_array=sieve(number, boundary, factor_base)
print(sieved_array)

[1, 16]
4.0000


IndexError: list index out of range

In [36]:
def __sieve_interval (number , factor_base , start_point , end_point , threshold ) :
    
    sieving_array_length = end_point - start_point + 1
    logarithms=[]

    for prime in factor_base[3:]:
        square_roots = mod(number , prime ).sqrt( all = True )
        print(square_roots)
        found = 0
        logarithm = round( log ( prime , 2) ).n(digits=5)
        print(logarithm)

        for index in islice( count (0) , prime ) : 
            if ( start_point + index )%prime == square_roots [ 0 ] :
                for multiple in islice( count ( index , prime ) , ( sieving_array_length - index + prime - 1) //prime ) :
                    logarithms [ multiple ] += logarithm
                found += 1

            if ( start_point + index )%prime == square_roots [ 1 ] :
                for multiple in islice( count ( index , prime ) ,( sieving_array_length - index + prime - 1) //prime ) :
                    logarithms [ multiple ] += logarithm
                found += 1

            if found == 2: break

    sieved_array = [ ]
    for index in islice( count (0) , sieving_array_length ):

        if logarithms[index] >= threshold :
            composite = ( start_point + index ) **2 - number
            factorization = [ ]
            if composite < 0:
                composite *= -1
                factorization.append(-1)
            for prime in factor_base [1:] :
                while composite%prime == 0:
                    composite /= prime
                    factorization.append ( prime )
            if composite == 1:
                sieved_array.append ( [ start_point + index , factorization ] )
    return sieved_array

In [8]:
def generate_matrix ( exponent_vectors ):
    return matrix (GF(2) , exponent_vectors ).kernel().basis()

In [9]:
def factorize(number , sieved_array , matrix ) :
    for row in matrix :
        x = [ ]
        y = [ ]
        for index in islice( count (0) , len (row) ) :
            if row [ index ] == 1:
                x.append( sieved_array [ index ] [ 0 ] )
                y.append( sieved_array [ index ] [ 1 ] ) 
        y = sorted( reduce(lambda x , y : x + y , y) ) [ : : 2 ]
        y = reduce(lambda i , j : i *j , y)
        x = reduce(lambda i , j : i *j , x)
        factor = gcd (( x%number) - (y%number) , number)
        if ( factor != 1) and ( factor != number) :
            return [ factor , number/ factor ]
    return [ ]